In [1]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\hp\Desktop\internship\genai\.env_genai\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install langchain_community

  Using cached SQLAlchemy-2.0.29-cp310-cp310-win_amd64.whl (2.1 MB)
  Using cached PyYAML-6.0.1-cp310-cp310-win_amd64.whl (145 kB)
  Using cached multidict-6.0.5-cp310-cp310-win_amd64.whl (28 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl (5.7 kB)
  Using cached frozenlist-1.4.1-cp310-cp310-win_amd64.whl (50 kB)
  Using cached yarl-1.9.4-cp310-cp310-win_amd64.whl (76 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
  Using cached packaging-23.2-py3-none-any.whl (53 kB)
  Using cached jsonpointer-2.4-py2.py3-none-any.whl (7.8 kB)
  Using cached orjson-3.10.1-cp310-none-win_amd64.whl (139 kB)
  Using cached greenlet-3.0.3-cp310-cp310-win_amd64.whl (292 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\hp\Desktop\internship\genai\.env_genai\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
from langchain_community.document_loaders import PyPDFLoader

In [4]:
pip install -U langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\hp\Desktop\internship\genai\.env_genai\Scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
loader=PyPDFLoader(r"C:\Users\hp\Desktop\internship\genai\Rag_systems\2404.07143.pdf")
pages=loader.load_and_split()

In [7]:
from langchain_text_splitters import NLTKTextSplitter

text_splitter = NLTKTextSplitter(chunk_size=500,chunk_overlap=100)

chunks=text_splitter.split_documents(pages)

print(len(chunks))

print(type(chunks[0]))

Created a chunk of size 568, which is longer than the specified 500
Created a chunk of size 506, which is longer than the specified 500
Created a chunk of size 633, which is longer than the specified 500


110
<class 'langchain_core.documents.base.Document'>


In [8]:
chunks[0]

Document(page_content='Preprint.\n\nUnder review.\n\nLeave No Context Behind:\nEfficient Infinite Context Transformers with Infini-attention\nTsendsuren Munkhdalai, Manaal Faruqui and Siddharth Gopal\nGoogle\ntsendsuren@google.com\nAbstract\nThis work introduces an efficient method to scale Transformer-based Large\nLanguage Models (LLMs) to infinitely long inputs with bounded memory\nand computation.\n\nA key component in our proposed approach is a new at-\ntention technique dubbed Infini-attention.', metadata={'source': 'C:\\Users\\hp\\Desktop\\internship\\genai\\Rag_systems\\2404.07143.pdf', 'page': 0})

In [9]:
pip install langchain-google-genai

You should consider upgrading via the 'c:\Users\hp\Desktop\internship\genai\.env_genai\Scripts\python.exe -m pip install --upgrade pip' command.


In [16]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
f = open(r"C:\Users\hp\Desktop\internship\genai\keys\gemini_key.txt")
api_key = f.read()
embedding_model = GoogleGenerativeAIEmbeddings(google_api_key=api_key ,
                                               model="models/embedding-001")

In [17]:
# Store the chunks in vector store
from langchain_community.vectorstores import Chroma

# Embed each chunk and load it into the vector store
db = Chroma.from_documents(chunks, embedding_model, persist_directory="./chroma_db_")

# Persist the database on drive
db.persist()

In [18]:
# Setting a Connection with the ChromaDB
connection = Chroma(persist_directory="./chroma_db_", embedding_function=embedding_model)

In [19]:
# Converting CHROMA db_connection to Retriever Object
retriever = connection.as_retriever(search_kwargs={"k": 5})

print(type(retriever))

<class 'langchain_core.vectorstores.VectorStoreRetriever'>


In [20]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [21]:
chat_template = ChatPromptTemplate.from_messages([
    # System Message Prompt Template
    SystemMessage(content="""You are a Helpful AI Bot. 
    Your task is to provide assistance based on the context given by the user. 
    Make sure your answers are relevant and helpful."""),
    # Human Message Prompt Template
    HumanMessagePromptTemplate.from_template("""Answer the question based on the given context.
    Context:
    {context}
    
    Question: 
    {question}
    
    Answer: """)
])

In [22]:
from langchain_google_genai import ChatGoogleGenerativeAI

chat_model = ChatGoogleGenerativeAI(google_api_key=api_key, 
                                   model="gemini-1.5-pro-latest")

In [23]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [24]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | chat_template
    | chat_model
    | output_parser
)

In [25]:
response = rag_chain.invoke("what is gating score visualization")

response

'## Gating Score Visualization Explained\n\nBased on the context provided, **gating score visualization** refers to the graphical representation of the gating scores (sigmoid(β)) within the compressive memory of the Infini-attention model. This visualization helps analyze the behavior of different attention heads in each layer of the model. \n\n**Figure 3** specifically showcases this visualization, revealing two distinct types of attention heads that emerge after training:\n\n* **Specialized Heads:** These heads exhibit gating scores near 0 or 1, indicating a strong preference for either utilizing or bypassing the compressive memory.\n* **Mixer Heads:** These heads have gating scores close to 0.5, suggesting a more balanced approach where they combine information from both the compressive memory and the original input.\n\nAnalyzing the distribution and behavior of these heads through gating score visualization provides valuable insights into how the model utilizes its memory and atten

In [26]:
from IPython.display import Markdown as md

md(response)

## Gating Score Visualization Explained

Based on the context provided, **gating score visualization** refers to the graphical representation of the gating scores (sigmoid(β)) within the compressive memory of the Infini-attention model. This visualization helps analyze the behavior of different attention heads in each layer of the model. 

**Figure 3** specifically showcases this visualization, revealing two distinct types of attention heads that emerge after training:

* **Specialized Heads:** These heads exhibit gating scores near 0 or 1, indicating a strong preference for either utilizing or bypassing the compressive memory.
* **Mixer Heads:** These heads have gating scores close to 0.5, suggesting a more balanced approach where they combine information from both the compressive memory and the original input.

Analyzing the distribution and behavior of these heads through gating score visualization provides valuable insights into how the model utilizes its memory and attention mechanisms to process information. 


In [27]:
response = rag_chain.invoke("Can you please tell me about meory retival and memory update")

response

'## Memory Retrieval and Update in Compressive Memory Systems:\n\nBased on the context, it seems we\'re discussing **compressive memory systems** used in the context of attention mechanisms (like those seen in transformer models). Let\'s break down memory retrieval and update processes:\n\n**Memory Retrieval:**\n\n* **Objective:** To efficiently fetch relevant information from the stored memory when processing new data.\n* **Mechanism:** \n    * The system utilizes the **attention query states** from the current input sequence as keys to access relevant information within the compressed memory.\n    * This is similar to how standard attention mechanisms work, but instead of attending over a fixed context window, it attends over the entire compressed memory.\n    * The system retrieves the most relevant "value" representations associated with the query, aiding in understanding the current input based on past information.\n\n**Memory Update:**\n\n* **Objective:** To efficiently store new

In [28]:
md(response)

## Memory Retrieval and Update in Compressive Memory Systems:

Based on the context, it seems we're discussing **compressive memory systems** used in the context of attention mechanisms (like those seen in transformer models). Let's break down memory retrieval and update processes:

**Memory Retrieval:**

* **Objective:** To efficiently fetch relevant information from the stored memory when processing new data.
* **Mechanism:** 
    * The system utilizes the **attention query states** from the current input sequence as keys to access relevant information within the compressed memory.
    * This is similar to how standard attention mechanisms work, but instead of attending over a fixed context window, it attends over the entire compressed memory.
    * The system retrieves the most relevant "value" representations associated with the query, aiding in understanding the current input based on past information.

**Memory Update:**

* **Objective:** To efficiently store new information in the compressed memory without unbounded growth in memory size.
* **Mechanism:**
    * **Compressive memory** maintains a fixed number of parameters, unlike traditional memory that grows with input.
    * New information is incorporated by **updating the memory parameters** through an optimization process. 
    * The objective of this update is to ensure that the new information can be accurately retrieved later when needed. 
    * This involves finding a compressed representation of the new information that minimizes information loss and maximizes the ability to recover it during retrieval.

**Benefits of this approach:**

* **Bounded Storage and Computation:** Memory usage and computational cost remain manageable even with long sequences, unlike standard attention where memory grows linearly with sequence length.
* **Efficient Retrieval:** The attention-based retrieval mechanism allows for quick and targeted access to relevant past information.
* **Leveraging Existing Techniques:** The system can utilize established training methods from attention mechanisms, aiding in stability and performance.

**Additional Notes:**

* The context mentions the work of Katharopoulos et al. (2020) as a basis for the update and retrieval mechanisms due to its simplicity and effectiveness. 
* The system described also reuses key and value states from standard attention computations for memory consolidation, making efficient use of computed information. 
